In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 드라이브 위치 이동
%cd -----PATH-----

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("----DATA----")

In [ ]:
data.columns

In [ ]:
data_need = data.drop(['membership_type_1', 'membership_type_2',
       'membership_type_3', 'membership_type_4', 'membership_type_5',
       'membership_type_6','tran_date','tran_time','play_date','play_st_time',
       'discount_type','pre_open_date','open_date'], axis=1)

In [ ]:
data_need = data_need[data_need.ticket_cancel==0]
data_need = data_need.drop(['ticket_cancel'],axis=1)
data_need = data_need[data_need.price > 0]
data_need

In [ ]:
import statsmodels.api as sm
import seaborn as sns

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data_need.isnull().sum()

In [ ]:
data_f = data_need.dropna(axis=0)
data_f.isnull().sum()

In [ ]:
data_e = data_f.copy()

In [ ]:
# sns.pairplot(data_e)
# plt.show()

In [ ]:
# plt.figure(figsize=(20, 12))
# plt.subplot(3,3,1)
# sns.boxplot(x='age', y='price', data=data_e)
# plt.subplot(3,3,2)
# sns.boxplot(x='gender', y='price', data=data_e)
# plt.subplot(3,3,3)
# sns.boxplot(x='seat', y='price', data=data_e)
# plt.subplot(3,3,4)
# sns.boxplot(x='performance_code', y='price', data=data_e)
# plt.subplot(3,3,5)
# sns.boxplot(x='genre', y='price', data=data_e)
# plt.subplot(3,3,6)
# sns.boxplot(x='member_yn', y='price', data=data_e)
# plt.show()

In [ ]:
data_e.index = np.arange(len(data_e))

In [ ]:
data_e

In [ ]:
# 성별 -> Male : 0, Female : 1
def toNumeric_gender(x):
    return x.map({"M":0, "F":1})

data_e[['gender']] = data_e[['gender']].apply(toNumeric_gender)
data_e

In [ ]:
data_e.place.unique()

In [ ]:
# place -> 0 ~ N으로
def toNumeric_place(x):
    return x.map({"콘서트홀":0, "IBK챔버홀":1, "리사이틀홀":2})

data_e[['place']] = data_e[['place']].apply(toNumeric_place)
data_e

In [ ]:
data_e.genre.unique()

In [ ]:
# genre -> 0 ~ N으로
def toNumeric_genre(x):
    return x.map({"독주":0,
                  "오페라":1,
                  "합창":2,
                  "클래식":3,
                  "교향곡":4,
                  "실내악":5,
                  "콘서트":6,
                  "성악":7,
                  "복합장르":8,
                  "재즈":9,
                  "크로스오버":10,
                  "기타":11,
                  "가족극":12,
                  "전시":13,
                  "무용":14})

data_e[['genre']] = data_e[['genre']].apply(toNumeric_genre)
data_e

In [ ]:
# seat -> 1층, 2층, 3층, 합창으로(일단 간단하게)
def toNumeric_seat(x):
    if "1층" in x : return 0
    elif "2층" in x : return 1
    elif "3층" in x : return 2
    elif "합창석" in x : return 3
    else : return 4

data_e['seat'] = data_e['seat'].map(toNumeric_seat)
data_e

In [ ]:
# member_yn -> 0 ~ N으로
def toNumeric_membership(x):
    return x.map({"N":0, "Y":1})

data_e[['member_yn']] = data_e[['member_yn']].apply(toNumeric_membership)
data_e

In [ ]:
Y = data_e.price

In [ ]:
X = data_e[['age','gender','seat','performance_code','genre','place','running_time','intermission','member_yn']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

def preprocessing(X):
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    variables = X_scaled
    vif = pd.DataFrame()
    vif["VIF"] = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
    vif["Features"] = X.columns
    print(vif)

In [ ]:
preprocessing(X)

In [ ]:
X.drop(['running_time', 'intermission', 'member_yn'], axis=1, inplace=True)
preprocessing(X)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.25,random_state=355)

## Multiple Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
regression = LinearRegression()
regression.fit(x_train,y_train)

In [ ]:
y_predict = regression.predict(x_test)

In [ ]:
plt.scatter(y_test,y_predict)
plt.suptitle('y_test vs y_pred', fontsize=20)
plt.xlabel('y_test', fontsize=18)
plt.ylabel('y_pred', fontsize=16)

In [ ]:
pred = pd.DataFrame(y_predict)
pred.columns = ['price_pred']
pred

In [ ]:
yt = pd.DataFrame(y_test)
yt.index = np.arange(len(yt))
yt.columns = ['price_true']
yt

In [ ]:
yy = pd.concat([yt, pred], axis=1)
yy['loss'] = yt.price_true - pred.price_pred
yy
# loss = yt.price_true.tolist()-pred.price_pred.tolist()
# yy['loss'] = loss
# yy

In [ ]:
plt.plot(yy['price_true'][0:100], color='red')
plt.plot(yy['price_pred'][0:100], color='green')
plt.show()

In [ ]:
import statsmodels.api as sms
model_1 = sms.OLS(y_train, x_train).fit()
model_1.summary()

## KNN

In [ ]:
from sklearn import neighbors
from sklearn.metrics import mean_squared_error
from math import sqrt
%matplotlib inline

rmse_val = []
for K in range(20):
    K = K+1
    model = neighbors.KNeighborsRegressor(n_neighbors = K)

    model.fit(x_train, y_train)  #fit the model
    pred=model.predict(x_test) #make prediction on test set
    error = sqrt(mean_squared_error(y_test,pred)) #calculate rmse
    rmse_val.append(error) #store rmse values
    print('RMSE value for k= ' , K , 'is:', error)

In [ ]:
curve = pd.DataFrame(rmse_val) #elbow curve
curve.plot()

In [ ]:
# K = 7일때 가장 error가 작음
model = neighbors.KNeighborsRegressor(n_neighbors = K)
model.fit(x_train, y_train)  #fit the model
pred=model.predict(x_test) #make prediction on test set

In [ ]:
true_y = pd.DataFrame(y_test)
true_y.columns = ['price_true']
true_y.index = np.arange(len(true_y))

In [ ]:
pred_d = pd.DataFrame(pred)
pred_d.columns = ['price_pred']
pred_d.index = np.arange(len(pred_d))

In [ ]:
yy_2 = pd.concat([true_y, pred_d], axis=1)
yy_2['loss'] = true_y.price_true - pred_d.price_pred
yy_2

In [ ]:
yy_2.loss.mean()

In [ ]:
yy.loss.mean()

In [ ]:
plt.plot(yy['price_true'][0:100], color='red')
plt.plot(yy['price_pred'][0:100], color='green')
plt.show()